# Стоимость жилья в Калифорнии. 

**Содержание:**  
1. [Импорт данных и общая информация](#load_and_info)  
 1.1. [Импорт библиотек](#import)  
 1.2. [Импорт данных](#read)  
 1.3. [Общая информация](#info)  
    
    
2. [Предобработка данных](#preparation)  
 
 
3. [Построение моделей прогнозирования](#modeling)  

 
4. [Проверка гипотезы о сопоставимости качества предсказаний моделей](#hip)  


5. [Общий вывод](#final)  

**Цель исследования:** предсказать медианную стоимость дома в жилом массиве.

**Задачи:**  
1. Обучить модель и сделать предсказания на тестовой выборке.
2. Оценить качество моделей с помощью метрик R2, RMSE и MAE.

**Исходные данные:**  
Исходные данные представлены в датасете housing.csv - данные о жилье в Калифорнии в 1990 году.

В датасете содержатся следующие признаки:  
- longitude — широта;
- latitude — долгота;
- housing_median_age — медианный возраст жителей жилого массива;
- total_rooms — общее количество комнат в домах жилого массива;
- total_bedrooms — общее количество спален в домах жилого массива;
- population — количество человек, которые проживают в жилом массиве;
- households — количество домовладений в жилом массиве;
- median_income — медианный доход жителей жилого массива;
- median_house_value — медианная стоимость дома в жилом массиве;
- ocean_proximity — близость к океану.

## Импорт данных и общая информация
<a id="load_and_info"></a>

### Импорт библиотек
<a id="import"></a>

In [1]:
import pandas as pd 
import numpy as np
import warnings

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F

from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler, Imputer
from pyspark.ml.regression import LinearRegression
from pyspark.mllib.evaluation import RegressionMetrics
from pyspark.ml.evaluation import RegressionEvaluator

pyspark_version = pyspark.__version__
if int(pyspark_version[:1]) == 3:
    from pyspark.ml.feature import OneHotEncoder         as OHE
elif int(pyspark_version[:1]) == 2:
    from pyspark.ml.feature import OneHotEncodeEstimator as OHE
    
from scipy.stats import levene
from scipy import stats as st
    
warnings.filterwarnings("ignore")

### Импорт данных
<a id="read"></a>

In [4]:
RANDOM_SEED = 2022

spark = SparkSession.builder \
                    .master("local") \
                    .appName("Titanic - Linear regression") \
                    .getOrCreate()

df = spark.read.option('header', 'true').csv('/datasets/housing.csv', inferSchema = True)

### Общая информация
<a id="info"></a>

Выводим схему таблицы:

In [5]:
df.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)



Определяем количество строк в датафрейме:

In [6]:
print('Количество строк в датафрейме:', df.count())

Количество строк в датафрейме: 20640


Подсчитываем количество пропущенных значений:

In [7]:
Dict_Null = {col:df.filter(df[col].isNull()).count() for col in df.columns}
Dict_Null

{'longitude': 0,
 'latitude': 0,
 'housing_median_age': 0,
 'total_rooms': 0,
 'total_bedrooms': 207,
 'population': 0,
 'households': 0,
 'median_income': 0,
 'median_house_value': 0,
 'ocean_proximity': 0}

In [8]:
sum_nan = 0
subset = []
for key, value in Dict_Null.items():
    sum_nan += value
    if value != 0:
        subset.append(key)
print('Максимальный процент строк с пропущенными значениями: {:.2%}'.format(sum_nan/df.toPandas().shape[0]))

Максимальный процент строк с пропущенными значениями: 1.00%


Процент строк с пропущенными значениями невелик - 1% (все для признака "total_bedrooms").

**Вывод:**  
1. Датафрейм состоит из 20640 строк и 10 признаков.  
2. В датафрейме 207 пропущенных значений.  
3. Cледующие признаки имеют тип double:  
 - longitude;  
 - latitude;  
 - housing_median_age;  
 - total_rooms;  
 - total_bedrooms;  
 - population;  
 - households;  
 - median_income;  
 - median_house_value.  

 Cледующие признаки имеют тип string:  
 - ocean_proximity.

## Предобработка данных
<a id="preparation"></a>  

Разбиваем категориальные, числовые данные и целевой признак на соответствующие списки:

In [9]:
categorical_cols = ['ocean_proximity']
numerical_cols  = ['longitude', 'latitude', 'housing_median_age', 'total_rooms', 
                  'total_bedrooms','population', 'households', 'median_income']
target = 'median_house_value'

Заменяем пропущенные значения числовых признаков на медианные значения, а текстовых - "no available":

In [10]:
for i in numerical_cols:
    df = df.na.fill(df.agg(F.expr('percentile(' + i + ', 0.5)')).collect()[0][0], i)
    
for i in categorical_cols:
    df = df.na.fill('no available', i)
    
df.describe().toPandas()

,summary,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,count,20640,20640,20640,20640,20640,20640,20640,20640,20640,20640
1,mean,-119.56970445736148,35.6318614341087,28.639486434108527,2635.7630813953488,536.8388565891473,1425.4767441860465,499.5396802325581,3.8706710029070246,206855.81690891474,None
2,stddev,2.003531723502584,2.135952397457101,12.58555761211163,2181.6152515827944,419.39187792168866,1132.46212176534,382.3297528316098,1.899821717945263,115395.61587441359,None
3,min,-124.35,32.54,1.0,2.0,1.0,3.0,1.0,0.4999,14999.0,<1H OCEAN
4,max,-114.31,41.95,52.0,39320.0,6445.0,35682.0,6082.0,15.0001,500001.0,NEAR OCEAN


**Вывод**  
1. Удалены пропущенные значения;  
2. Категориальные, числовые данные и целевой признак разбили на соответствующие списки.

## Построение моделей прогнозирования
<a id="modeling"></a>

Функуия построения моделей прогнозирования и рассчета метрик:

In [11]:
def modern(df_tmp, numerical_cols, categorical_cols=[]):
 
    
    # Разделяем датафрейм на обучающую и тестовую выборки
    train_data, test_data = df_tmp.randomSplit([.8,.2], seed=RANDOM_SEED)
    data_list = [train_data, test_data]
    
    if categorical_cols != []:
        # трансформируем категориальные признаки - переводим текстовые категории в числовое представление.
        indexer = StringIndexer(inputCols=categorical_cols, 
                            outputCols=[c+'_idx' for c in categorical_cols])
        indexer_tmp = indexer.setHandleInvalid("skip").fit(train_data)
        train_data = indexer_tmp.transform(train_data)
        test_data = indexer_tmp.transform(test_data)
        for i in data_list:
            cols = [c for c in i.columns for j in categorical_cols if (c.startswith(j))]
    
    
        #Производим OHE-кодирование для категориальных признаков
        encoder = OHE(inputCols=[c+'_idx' for c in categorical_cols],
                                outputCols=[c+'_ohe' for c in categorical_cols])
        encoder_tmp = encoder.fit(train_data)
        train_data = encoder_tmp.transform(train_data)
        test_data = encoder_tmp.transform(test_data)
        for i in data_list:
            cols = [c for c in i.columns for j in categorical_cols if (c.startswith(j))]
        
        
        # Объединяем категориальные признаки в один вектор
        categorical_assembler = VectorAssembler(inputCols=[c+'_ohe' for c in categorical_cols],
                                                outputCol="categorical_features")
        train_data = categorical_assembler.transform(train_data)
        test_data = categorical_assembler.transform(test_data)
    
    
    # Объединяем числовые признаки в один вектор
    numerical_assembler = VectorAssembler(inputCols=numerical_cols,
                                          outputCol="numerical_features")
    train_data = numerical_assembler.transform(train_data)
    test_data = numerical_assembler.transform(test_data)
    
    
    # Масштабируем числовые признаки
    standardScaler = StandardScaler(inputCol='numerical_features',
                                    outputCol="numerical_features_scaled")
    standardScaler_tmp = standardScaler.fit(train_data)
    train_data = standardScaler_tmp.transform(train_data)
    test_data = standardScaler_tmp.transform(test_data)
    
    
    # собраем трансформированные категорийные и числовые в один вектор
    if categorical_cols != []:
        all_features = ['categorical_features','numerical_features_scaled']
    else:
        all_features = ['numerical_features_scaled']
    final_assembler = VectorAssembler(inputCols=all_features, 
                                      outputCol="features") 
    train_data = final_assembler.transform(train_data)
    test_data = final_assembler.transform(test_data)
    
    
    # Обучаем модель LinearRegression
    lr = LinearRegression(labelCol=target, featuresCol='features')
    model = lr.fit(train_data) 
    
    
    # Осуществляем предсказание целевого признака
    predictions = model.transform(test_data)
    predictedLabes = predictions.select("median_house_value", "prediction")
    
    
    # Рассчитываем метрику R2
    r2_metric = RegressionEvaluator(predictionCol="prediction", labelCol="median_house_value",metricName="r2")
    print('R2: {:.3f}'.format(r2_metric.evaluate(predictions)))
    
    
    # Рассчитываем метрику RMSE
    rmse_metric = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="median_house_value",metricName="rmse")
    print('RMSE: {:.3f}'.format(rmse_metric.evaluate(predictions)))
    
    # Рассчитываем метрику MAE
    mae_metric = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="median_house_value",metricName="mae")
    print('MAE: {:.3f}'.format(mae_metric.evaluate(predictions)))
    
    return predictedLabes.toPandas()

Метрики R2, RMSE и MAE для модели, построенной по всем данным:

In [12]:
predict_all = modern(df, numerical_cols, categorical_cols)

23/09/30 11:43:44 WARN Instrumentation: [17f7464f] regParam is zero, which might cause numerical instability and overfitting.
23/09/30 11:43:44 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/09/30 11:43:44 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
23/09/30 11:43:45 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
23/09/30 11:43:45 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


R2: 0.656
RMSE: 68223.129
MAE: 49725.965


Метрики R2, RMSE и MAE для модели, построенной по числовым данным:

In [13]:
predict_num = modern(df, numerical_cols)

23/09/30 11:43:51 WARN Instrumentation: [1c974f1f] regParam is zero, which might cause numerical instability and overfitting.


R2: 0.646
RMSE: 69212.861
MAE: 50866.560


Останавливать сессию spark:

In [14]:
spark.stop()

**Вывод:**  
1. Модель LinearRegression обучена как на всех признаках, так и только на числовых.  
2. Метрики модели, обученной на всех данных:  
 - R2:  0.656;  
 - RMSE:  68223.129;  
 - MAE:  49725.965;  
 
 Метрики модели, обученной на числовых данных:  
 - R2:  0.646;  
 - RMSE:  69212.861;  
 - MAE:  50866.560;  
 
 
 Метрики обоих моделей сопоставимы. При этом значение R2 выше у модели,  обученной на всех данных, а RMSE и  MAE - у модели, обученной только на числовых значениях.

## Общий вывод
<a id="final"></a>

**Импорт данных и общая информация**  
1. Датафрейм состоит из 20640 строк и 10 признаков.  
2. В датафрейме 207 пропущенных значений.  
3. Cледующие признаки имеют тип double:  
 - longitude;  
 - latitude;  
 - housing_median_age;  
 - total_rooms;  
 - total_bedrooms;  
 - population;  
 - households;  
 - median_income;  
 - median_house_value.  

 Cледующие признаки имеют тип string:  
 - ocean_proximity.
 

**Подготовка данных**  
1. Удалены пропущенные значения;  
2. Категориальные, числовые данные и целевой признак разбили на соответствующие списки.  


**Построение моделей прогнозирования**  
1. Модель LinearRegression обучена как на всех признаках, так и только на числовых.  
2. Метрики модели, обученной на всех данных:  
 - R2:  0.656;  
 - RMSE:  68223.129;  
 - MAE:  49725.965;  
 
 Метрики модели, обученной на числовых данных:  
 - R2:  0.646;  
 - RMSE:  69212.861;  
 - MAE:  50866.560;  
 
 
 Метрики обоих моделей сопоставимы. При этом значение R2 выше у модели,  обученной на всех данных, а RMSE и  MAE - у модели, обученной только на числовых значениях. 